installer selenium et pandas 

In [2]:
!pip install selenium pandas


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Verification des paquets installer

In [47]:
!pip freeze
!pip freeze > requierements.txt

anyio==4.10.0
argon2-cffi==25.1.0
argon2-cffi-bindings==25.1.0
arrow==1.3.0
asttokens==3.0.0
async-lru==2.0.5
attrs==25.3.0
Automat==25.4.16
babel==2.17.0
beautifulsoup4==4.13.5
bleach==6.2.0
certifi==2025.7.14
cffi==1.17.1
charset-normalizer==3.4.3
colorama==0.4.6
comm==0.2.3
constantly==23.10.4
cryptography==45.0.7
cssselect==1.3.0
debugpy==1.8.16
decorator==5.2.1
defusedxml==0.7.1
elastic-transport==8.17.1
elasticsearch==8.13.0
executing==2.2.1
fastjsonschema==2.21.2
filelock==3.19.1
fqdn==1.5.1
h11==0.16.0
httpcore==1.0.9
httpx==0.28.1
hyperlink==21.0.0
idna==3.10
incremental==24.7.2
ipykernel==6.30.1
ipython==9.5.0
ipython_pygments_lexers==1.1.1
isoduration==20.11.0
itemadapter==0.12.2
itemloaders==1.3.2
jedi==0.19.2
Jinja2==3.1.6
jmespath==1.0.1
json5==0.12.1
jsonpointer==3.0.0
jsonschema==4.25.1
jsonschema-specifications==2025.4.1
jupyter-events==0.12.0
jupyter-lsp==2.3.0
jupyter_client==8.6.3
jupyter_core==5.8.1
jupyter_server==2.17.0
jupyter_server_terminals==0.5.3
jupyterlab=

In [34]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
import time
import json

# chemin vers chromedriver.exe
chrome = Service(r'C:/Program Files/chromedriver-win64/chromedriver.exe')

# fichier cookies facebook
facebook_cookies = r'C:/Users/Lenovo/nlpProjects/cookies.json'

# charger les cookies
def load_cookies(driver, facebook_cookies):
    with open(facebook_cookies, 'r') as file:
        cookies = json.load(file)
        for cookie in cookies:
            # certains champs ne sont pas acceptés par Selenium
            cookie.pop('sameSite', None)
            cookie.pop('priority', None)
            try:
                driver.add_cookie(cookie)
            except Exception as e:
                print(f"Cookie ignoré : {cookie} | Erreur : {e}")

# Initialiser le WebDriver pour Chrome
driver = webdriver.Chrome(service=chrome)

try:
    # charger url de la page d'accueil facebook
    driver.get('https://web.facebook.com/')

    # Charger les cookies pour maintenir la session
    load_cookies(driver, facebook_cookies)

    # Rafraîchir la page pour appliquer les cookies
    driver.refresh()

    # Attendre que la page se charge complètement
    time.sleep(5)

    # Aller sur la page cible
    driver.get("https://web.facebook.com/uvburkina")
    time.sleep(5)

    # Faire défiler pour charger tous les commentaires
    scroll_pause_time = 2
    max_scrolls = 10  # Éviter une boucle infinie
    scroll_count = 0
    
    while scroll_count < max_scrolls:
        last_height = driver.execute_script("return document.body.scrollHeight")
        driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.END)
        time.sleep(scroll_pause_time)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        scroll_count += 1

    # Extraire les commentaires
    comments = []
    try:
        # cibler uniquement les zones de texte dans les commentaires
        comment_elements = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.XPATH, "//div[@class='xdj266r x14z9mp xat24cr x1lziwak x1vvkbs']"))
        )
        
        for comment in comment_elements:
            texte = comment.text.strip()
            if not texte:
                continue

            # --- FILTRAGE ---
            # ignorer boutons
            if texte.lower() in ["j’aime", "répondre", "partager", "envoyer", "ajouter", "like", "reply", "share"]:
                continue

            # ignorer les courts textes
            if len(texte.split()) < 3:
                continue  # CORRECTION : "continues" -> "continue"

            # ignorer si ça ressemble à un lien/publication
            if "http" in texte.lower() or "www." in texte.lower():
                continue

            # ignorer les textes trop longs (probablement des publications)
            if len(texte) > 500:
                continue

            # garder uniquement les textes filtrés
            comments.append(texte)

    except Exception as e:
        print(f"Erreur lors de l'extraction des commentaires: {e}")

    # Afficher les résultats
    print(f"Nombre de commentaires extraits : {len(comments)}")
    for i, commentaire in enumerate(comments, 1):
        print(f"{i}. {commentaire}")

    # Sauvegarder dans un fichier JSON
    with open("commentaires.json", "w", encoding="utf-8") as fichier:
        json.dump(comments, fichier, ensure_ascii=False, indent=2)
    

except Exception as e:
    print(f"Erreur générale : {e}")

finally:
    # Fermer le navigateur
    driver.quit()

Nombre de commentaires extraits : 2
1. Nous bobolais on réclame pour nous
2. Bonsoir
Université Virtuelle du Burkina Faso - UVBF
, svp j’aimerais savoir si les candidature pour le master en fouille de donnée et intelligence artificielle de l’année 2025-2026 est lancée svp
